<a href="https://colab.research.google.com/github/Mackenzzy/machinelearning/blob/main/estudoAprendizagemDeMaquina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#instalação da lib e abrindo seção no spark
!pip install pyspark==3.3.1
from pyspark.sql import SparkSession

sc = SparkSession \
    .builder \
    .master('local[*]') \
    .getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 16.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845493 sha256=fad6d4d1daf174ffa744dc46877102655701ee58cdc7f5e936be8601d20eb4d3
  Stored in directory: /root/.cache/pip/wheels/0f/f0/3d/517368b8ce80486e84f89f214e0a022554e4ee64969f46279b
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.3
    Uninstalling pyspark-3.5.3:
      Successfully uninstalled pyspark-3.5.3


In [2]:
#download do db sobre o covid, fiz separado pois é um db gigante

!gdown 1-lSo7A-SIuntho9tAqVO25enmLlxFe7S
!gdown 1-iG4e4-SzJONLi3Nw98nm9UdFzoqCHDI

Downloading...
From: https://drive.google.com/uc?id=1-lSo7A-SIuntho9tAqVO25enmLlxFe7S
To: /content/treinamento.csv
100% 26.1M/26.1M [00:00<00:00, 47.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-iG4e4-SzJONLi3Nw98nm9UdFzoqCHDI
To: /content/teste.csv
100% 37.4M/37.4M [00:00<00:00, 56.9MB/s]


In [5]:
# criando os dados para servir de treino
dfTreino = sc.read\
    .option('delimiter',',')\
    .option('header','true')\
    .option('inferschema','true')\
    .csv('file:////content/treinamento.csv')

# criando os dados para ser usados no teste
dfTeste = sc.read\
    .option('delimiter',',')\
    .option('header','true')\
    .option('inferschema','true')\
    .csv('file:////content/teste.csv')

In [7]:
#nome da coluna \ tipo do dado

dfTreino.printSchema()

root
 |-- hora: double (nullable = true)
 |-- minuto: double (nullable = true)
 |-- temp_minima: double (nullable = true)
 |-- temp_maxima: double (nullable = true)
 |-- latitude_media: double (nullable = true)
 |-- longitude_media: double (nullable = true)
 |-- Classe: string (nullable = true)



In [15]:
#funcao para os id's
from pyspark.sql.functions import monotonically_increasing_id
#como a tabela não apresentou a coluna id(padrão no powerbi) eu adicionei
#para vir no indice 1 ao invés do 0 precisa ser colocado o '+1' depois dos ()
dfTeste = dfTeste.withColumn('id', monotonically_increasing_id() + 1)

#mostra as N primeiras linhas
dfTeste.show(8)

+---------+---------+-----------+-----------+--------------+---------------+--------+---+
|     hora|   minuto|temp_minima|temp_maxima|latitude_media|longitude_media|  Classe| id|
+---------+---------+-----------+-----------+--------------+---------------+--------+---+
|11.787212|29.005116|      1.123|     28.606|      34.56035|      136.21983|Moderado|  1|
|11.482636| 29.84576|      1.038|  29.306004|      34.56078|      136.22015|Moderado|  2|
|11.312064|29.751745|      1.123|  29.006002|      34.55864|      136.22047|Moderado|  3|
|11.530982|29.505793|      1.404|  29.081003|     34.560654|      136.22006|Moderado|  4|
|  11.4008|29.422289|      1.504|  29.181004|     34.559814|      136.22006|Moderado|  5|
| 11.50784| 29.18513|      1.423|  29.106003|     34.559834|      136.21964|Moderado|  6|
|11.318713|29.743177|      1.323|  28.806002|     34.560326|       136.2207|Moderado|  7|
|11.324524| 30.05444|      1.155|  29.006002|     34.560883|      136.21967|Moderado|  8|
+---------